In [1]:
import pandas as pd
import numpy as np
import datetime

import json
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
%matplotlib inline

# import seaborn as sns
# plt.style.use('ggplot')
# plt.figure(figsize=(16,4))

import warnings
warnings.filterwarnings("ignore")

from collections import Counter


In [2]:
df_users_eng = pd.read_csv('takehome_user_engagement.csv')

In [3]:
df_users_eng.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [4]:
df_users = pd.read_csv('takehome_users.csv',encoding='latin-1')

In [5]:
df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [6]:
# Checking both the dataframes whether they have same nuuber of users or not.

print('Number of users in df_users', (df_users.object_id.nunique()))
print('Number of users in df_users_eng', (df_users_eng.user_id.nunique()))

Number of users in df_users 12000
Number of users in df_users_eng 8823


In [7]:
# We can see, there are around 3K users id missing in df_users_eng.
# Precisely, there are 3177 users who have created the account but never used the application. 
# Thus, data is unavailable in df_users data frame

In [8]:
# Lets make the 'Time stamp' field as an index. it would be easy to analyse the data further.

df_users_eng.time_stamp = pd.to_datetime(df_users_eng.time_stamp)

# df_users_eng.set_index('time_stamp',inplace = True)

In [9]:
# Most of the analysis is going to be based on time.
# Thus, we need to convert all time/date related column into datetime format

In [10]:
df_users.creation_time = pd.to_datetime(df_users.creation_time)
#df_users.last_session_creation_time = pd.to_datetime(df_users.last_session_creation_time,unit='s')
df_users['last_session_creation_time'] = pd.to_datetime(df_users['last_session_creation_time'], unit='s')

In [11]:
df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0


Let's start with finding the adopted users.

In [12]:

time_delta_7 = datetime.timedelta(7)
adopted_users_dict = {}

temp1 = sorted(list(df_users_eng['user_id'].unique()))

for user_id in temp1:
    adopted_user = False
    temp = df_users_eng[df_users_eng['user_id'] == user_id]
    temp = temp.sort_values('time_stamp')
    
    for row in temp.itertuples():
        if adopted_user == True:
            continue
        time = temp.get_value(row[0], 'time_stamp')
        if len(temp[(temp['time_stamp'] >= time) & (temp['time_stamp'] <= (time + time_delta_7))]) >= 3:
            adopted_user = True
    
    adopted_users_dict[user_id] = adopted_user

In [ ]:
# Explore the user adoption labels
count_df = pd.DataFrame.from_dict(Counter(list(adopted_dict.values())), orient='index').reset_index()

print('We have analyzed logs for {} unique users.'.format(len(adopted_dict)))
print('{:.2f}% of users who have used the application are considered \'adopted users\'.'\
          .format(count_df.loc[1][1]/(count_df.loc[1][1]+count_df.loc[0][1])*100))